In [150]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime
from time import sleep
from urllib.error import HTTPError
import re
import json

team_names = {
    'ATL': 'ATL',
    'BOS': 'BOS',
    'BKN': 'BRK',
    'CHA': 'CHO',
    'CHI': 'CHI',
    'CLE': 'CLE',
    'DAL': 'DAL',
    'DEN': 'DEN',
    'DET': 'DET',
    'HOU': 'HOU',
    'IND': 'IND',
    'GS': 'GSW',
    'LAC': 'LAC',
    'LAL': 'LAL',
    'MEM': 'MEM',
    'MIA': 'MIA',
    'MIL': 'MIL',
    'MIN': 'MIN',
    'NO': 'NOP',
    'NY': 'NYK',
    'OKC': 'OKC',
    'ORL': 'ORL',
    'PHI': 'PHI',
    'PHX': 'PHO',
    'POR': 'POR',
    'SAC': 'SAC',
    'SA': 'SAS',
    'TOR': 'TOR',
    'UTAH': 'UTA',
    'WSH': 'WAS',
}


def process_game(soup, team):

    return


def scrape(home, away, d, game_number):
    day = d.strftime('%d')
    month = d.strftime('%m')
    year = d.strftime('%y')
    date = d.strftime('%Y-%m-%d')
    print(home, away, date)
    gameid = 21300000 + game_number
    gameid_str = '00%d' % gameid
    try:
        url = 'http://stats.inpredictable.com/nba/wpBox.php?season='+ year + '&month=' + month + '&date=' + date + '&gid=' + gameid_str
        url_page = urlopen(url)
    except HTTPError:
        sleep(10)
        url_page = urlopen(url)
    soup = BeautifulSoup(url_page, "html.parser")
    
    ### GET GAME excitement, comeback, mvp, lvp
    score_data = str(soup.find("div", {"id": "score"}))
    
    exc_text = 'Excitement</b></td><td>'
    exc_idx = score_data.index(exc_text)
    excitement_score = score_data[exc_idx + len(exc_text): exc_idx + len(exc_text) + 3]
    
    cb_text = 'Comeback</b></td><td>'
    cb_idx = score_data.index(cb_text)
    comeback_score = score_data[cb_idx + len(cb_text): cb_idx + len(cb_text) + 3]
    
    split = score_data.split('MVP</b></td><td>')[1]
    mvp = split.split('</td></tr>')[0]
    
    split = score_data.split('LVP</b></td><td>')[1]
    lvp = split.split('</td></tbody></table></div>')[0]
        
    ### GET EVENTS ###
    script = str(soup.findAll('script')[8])    
    split = script.split('"rows": ')[1]
    split = '{"rows": ' + split
    
    event_data = split.split('data_wprb.setColumnProperty')[0]
    event_data = event_data[:-23]
    events = []
    event_data = event_data.split('Q')
    for event in event_data:
        event = 'Q' + event
        event = event.replace('<u>', '').replace('</u>', '')
        event = event.replace('<b>', '').replace('</b>', '')
        event_split = event.split('<br />')
        if len(event_split) > 5:
            winprob_swing = event_split[2][-5:].lstrip()
            event = (event_split[1], winprob_swing)
            events.append(event)

    processed_game = "{},{},{},{},{},{},{},{}".format(home, away, d.strftime('%Y%m%d'), excitement_score, comeback_score, mvp, lvp, events)
    return processed_game
    
def process_year(year):

    game_list = open('data/'+year+'/'+year+'_articles.csv', 'r')
    file = open('data/'+year+'/'+year+'_play_by_play.csv', 'a')
    
    game_number = 0
    i = 0
    for game in game_list:
        i += 1
#         if i == 10:
#             break
        tokens = game.split(',')
        try:
            game_number += 1
            print("1", tokens[1])
            print("2",tokens[2])
            away = team_names[tokens[1]]
            home = team_names[tokens[2]]
            d = datetime.strptime(tokens[3], "%Y%m%d")
            game_data = scrape(home, away, d, game_number)
            file.write(game_data)
            sleep(2)
            file.write('\n')
        except KeyError:
            print("error")
            game_number -= 1
            continue
        except IndexError:
            print("index error")
            continue
    game_list.close()
    file.close()

process_year('2013')

1 away_team
2 home_team
error
1 ORL
2 IND
IND ORL 2013-10-29
1  Bird's alma mater. It was temporarily placed in the lobby of Bankers Life Fieldhouse. ... The Pacers lead the all-time series with Orlando 45-44. ... Indiana Hoosiers coach Tom Crean told Orlando's Fox affiliate that he would miss Oladipo's scoring and deflections more than his singing. ... Orlando's Tobias Harris and Doron Lamb sat out with sprained ankles. "

index error
1 CHI
2 MIA
MIA CHI 2013-10-29
1  Miami has won five straight games over the Bulls. ... James is 31 points shy of 25
2 000 for his career
error
1 LAC
2 LAL
LAL LAC 2013-10-29
1 BKN
2 CLE
CLE BRK 2013-10-30
1  the No. 1 overall pick in this year's draft
2  missed all five shots from the field. He finished with two points and five rebounds in 15 minutes. .... While Kidd serves his suspension
error
1 MIA
2 PHI
PHI MIA 2013-10-30
index error
1 BOS
2 TOR
TOR BOS 2013-10-30


KeyboardInterrupt: 